In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Build `states`
## Get margins of pf positions

In [ ]:
# Imports
import asyncio

import numpy as np
import pandas as pd
from ib_insync import MarketOrder

from utils import (clean_ib_util_df, get_a_margin, get_dte, get_margins,
                   get_order_pf, get_pickle,
                   qualify_conIds, qualify_me, to_list)

In [ ]:
# get unds, open orders and portfolio
unds = set(get_pickle(DATAPATH / 'unds.pkl').keys())
df_openorder, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
from utils import qualify_conIds
from ib_insync import Contract, ContractDetails
pf_contracts = asyncio.run(qualify_conIds(PORT, df_pf.conId))

In [ ]:
# Get margins of all positions
# ============================
CID = _vars.CID
desc = "Qualifiying Portfolio"
pf_contracts = to_list(df_pf.contract)

with IB().connect(port=PORT) as ib:
    pf_contracts = asyncio.run(qualify_me(ib, contracts=pf_contracts))

# ...integrate df_pf with multiplier
df1 = clean_ib_util_df(pf_contracts).set_index('conId')
df2 = df_pf.set_index('conId')

cols_to_use = df2.columns.difference(df1.columns)
df_pf = df1.join(df2[cols_to_use])

# join the multiplier
s = pd.to_numeric(df_pf.multiplier)
s.fillna(1, inplace=True)
df_pf = df_pf.assign(multiplier=s)

# Get DTEs
df_pf.insert(4, 'dte', df_pf.expiry.apply(lambda x: get_dte(x, MARKET)))
df_pf.loc[df_pf.dte <=0, "dte"] = 0

# Get the costPrice
df_pf.insert(9, 'costPrice', abs(df_pf.avgCost/df_pf.position))

# Assign the actions
df_pf = df_pf.assign(action=np.where(df_pf.position < 0, "BUY", "SELL"))

# build the orders
wif_order = [MarketOrder(action, totalQuantity) 
             for action, totalQuantity 
             in zip(df_pf.action, abs(df_pf.position).astype('int'))]
df_pf = df_pf.assign(wif_order = wif_order)

In [ ]:
with IB().connect(port=PORT) as ib:
    contracts = to_list(pf_contracts)
    orders = df_pf.wif_order.to_list()
    df_m = asyncio.run(get_margins(port=PORT, contracts=contracts, orders = orders))

In [ ]:
def join_my_df_with_another(my_df: pd.DataFrame, other_df: pd.DataFrame) -> pd.DataFrame:
    """
    Joins my df with other, protecting my columns
    my_df: original df
    other_df: df columns to be imported from

    Note:
    ---
    Index should be common across the dfs
    """
    cols_to_use = other_df.columns.difference(my_df.columns)
    df_out = my_df.join(other_df[cols_to_use])

    return df_out

In [ ]:
df_pfm = join_my_df_with_another(df_pf, df_m.set_index('conId')).drop(columns = ['wif_order'])

In [ ]:
df_pfm

In [ ]:
with IB().connect(port=PORT) as ib:
    asyncio.run(qualify_me(df_pfm.iloc[0].contract))

In [ ]:
from ib_insync import Contract

ct = Contract(secType='OPT', symbol='ABNB', lastTradeDateOrContractMonth='20240217', strike=124.0, right='P', exchange='SMART', currency='USD')
with IB().connect(port=PORT) as ib:
    ct = asyncio.run(qualify_me(ib, ct))

In [ ]:
from utils import qualify_me, qualify_conIds

with IB().connect(port=PORT) as ib:
    ct1_requested = asyncio.run(qualify_me(ib, [contracts[0]]))

ct1_qualified = asyncio.run(qualify_conIds(PORT=PORT, conIds=list([contracts[0].conId])))

In [ ]:
ct1_requested == ct1_qualified

# Unsowed

In [ ]:
# No orders to sow and no existing positions

options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'

long = df_pf.position > 0
long_options = long & options
df_pf[long_options]

unsowed = unds - set(df_openorder.symbol) - set(df_pf.symbol)
unsowed

# Orphaned

In [ ]:
# Long calls or puts without any underlying stock position
options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'
long = df_pf.position >= 1
long_stocks = long & stocks
long_options = long & options
orphaned = long_stocks & long_options

orphaned = df_pf[orphaned]
orphaned

# Unreaped

In [ ]:
# Open position without a reap order
df_reap = df_pf[options]